In [ ]:
# This sliding window protocol allows the sender to send multiple packets at a time 
# before waiting for an acknowledgement.

import socket
import time

# total packet size
PACKET_SIZE = 1024
SEQ_ID_SIZE = 4
MESSAGE_SIZE = PACKET_SIZE - SEQ_ID_SIZE
# window size: 100 packets
WINDOW_SIZE = 100

# read data
with open('docker/file.mp3', 'rb') as f:
    data = f.read()
    data = data[0:len(data)//30]

# create a udp socket
with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as udp_socket:

    # bind the socket to a OS port
    udp_socket.bind(("localhost", 5000))
    udp_socket.settimeout(1)

    # start sending data from 0th sequence
    base = 0
    next_seq_num = 0
    packets = {}

    start = time.time()
    while base < len(data):
        print(f"base: {base}")
        # send new packets if window is not full
        while next_seq_num < base + WINDOW_SIZE * MESSAGE_SIZE and next_seq_num < len(data):
            # construct and send message
            message = int.to_bytes(next_seq_num, SEQ_ID_SIZE, signed=True, byteorder='big') + data[next_seq_num : next_seq_num + MESSAGE_SIZE]
            udp_socket.sendto(message, ('localhost', 5001))
            # store the packet
            packets[next_seq_num] = message
            # move sequence id forward
            next_seq_num += MESSAGE_SIZE

        # wait for acknowledgement
        while True:
            try:
                # wait for ack and extract ack id
                ack, _ = udp_socket.recvfrom(PACKET_SIZE)
                ack_id = int.from_bytes(ack[:SEQ_ID_SIZE], byteorder='big')
                print(ack_id, ack[SEQ_ID_SIZE:])
                print(min(base + (WINDOW_SIZE+1) * MESSAGE_SIZE, len(data)))
                # ack id == base position, move on
                # last ack_id is len(data)
                if ack_id == min(base + (WINDOW_SIZE+1) * MESSAGE_SIZE, len(data)):
                    base = ack_id
                    break
                
            except socket.timeout:
                # no ack, resend all packets in the window
                print("resend")
                for seq_id, message in packets.items():
                    if seq_id >= base and seq_id < base + WINDOW_SIZE * MESSAGE_SIZE:
                        udp_socket.sendto(message, ('localhost', 5001))

    # send final closing message
    # send an empty message with the correct sequence id (seq_id + MESSAGE_SIZE)
    empty_message = int.to_bytes(len(data), 4, signed=True, byteorder='big')
    udp_socket.sendto(empty_message, ('localhost', 5001))
    while True:
            # wait for final ack
            final_ack, _ = udp_socket.recvfrom(PACKET_SIZE)
            # get the final message id
            seq_id, message = final_ack[:SEQ_ID_SIZE], final_ack[SEQ_ID_SIZE:]
            if message == b'ack':
                continue
            if message == b'fin':
                udp_socket.sendto(int.to_bytes(-1, 4, signed=True, byteorder='big') + '==FINACK=='.encode(), ('localhost', 5001))
                break
            
    end = time.time()
    print(f"throughput: {len(data)//(end-start)} bytes per seconds")
    print(f"time lapse: {(end-start)} seconds")



In [4]:
print(packets.keys())

dict_keys([0, 1020, 2040, 3060, 4080, 5100, 6120, 7140, 8160, 9180, 10200, 11220, 12240, 13260, 14280, 15300, 16320, 17340, 18360, 19380, 20400, 21420, 22440, 23460, 24480, 25500, 26520, 27540, 28560, 29580, 30600, 31620, 32640, 33660, 34680, 35700, 36720, 37740, 38760, 39780, 40800, 41820, 42840, 43860, 44880, 45900, 46920, 47940, 48960, 49980, 51000, 52020, 53040, 54060, 55080, 56100, 57120, 58140, 59160, 60180, 61200, 62220, 63240, 64260, 65280, 66300, 67320, 68340, 69360, 70380, 71400, 72420, 73440, 74460, 75480, 76500, 77520, 78540, 79560, 80580, 81600, 82620, 83640, 84660, 85680, 86700, 87720, 88740, 89760, 90780, 91800, 92820, 93840, 94860, 95880, 96900, 97920, 98940, 99960, 100980])
